In [117]:
%matplotlib inline

In [118]:
#export
from exp.nb_03 import *

# Data Setup

In [119]:
xTraining, yTraining, xValidation, yValidation = getMnistData()

In [120]:
xTrainingNormalized = normalizeVector(xTraining, xTraining.mean(), xTraining.std())
xValidationNormalized = normalizeVector(xValidation, xValidation.mean(), xValidation.std())

In [121]:
((xTrainingNormalized.std(), xTrainingNormalized.mean()), (xValidationNormalized.std(), xValidationNormalized.mean()))

((tensor(1.), tensor(-7.6999e-06)), (tensor(1.0000), tensor(-7.0751e-08)))

In [122]:
trainingDataSet = Dataset(xTrainingNormalized, yTraining)
validationDataSet = Dataset(xValidationNormalized, yValidation)

In [123]:
hiddenLayerOutput, batchSize = 50, 64

In [124]:
numberOfClasses = yValidation.max().item() + 1

In [125]:
trainingRows, trainingColumns = xTrainingNormalized.shape

In [126]:
learningRate = 0.5

In [127]:
selfLearningModel = SelfLearningLibraryModel(trainingColumns,hiddenLayerOutput,numberOfClasses,learningRate)

In [128]:
#export
class DataBunch():
    def __init__(self, trainingDataLoader, validationDataLoader):
        self.trainingDataLoader, self.validationDataLoader = trainingDataLoader, validationDataLoader
        
    @property
    def trainingDataSet(self): return self.trainingDataLoader
    
    @property
    def validationDataSet(self): return self.validationDataLoader

In [129]:
trainingDataLoader, validationDataLoader = createDataLoaders(trainingDataSet,
                                                            validationDataSet, 5000)

In [130]:
imageDataBunch = DataBunch(trainingDataLoader, validationDataLoader)

In [131]:
#export
def trainModelWithValidation(learnableModel, dataBunch, epochs):
    for epoch in range(epochs):
        for _xTrain, _yTrain in dataBunch.trainingDataLoader:
            _preds = learnableModel(_xTrain)
            loss = Functional.cross_entropy(_preds, _yTrain)
            loss.backward()
            learnableModel.learn()
        accumulatedLoss, accumulatedAccuracy = 0.,0.
        for _xValidation, _yValidation in dataBunch.validationDataLoader:
            with torch.no_grad(): # do not produce gradients when running loss function
                _preds = learnableModel(_xValidation)
                accumulatedLoss += Functional.cross_entropy(_preds, _yValidation)
                accumulatedAccuracy += accuracy(_preds, _yValidation)
        numberOfItems = len(validationDataLoader)
        print("Epoch {}, Accuracy {}, Loss {}".format(epoch, accumulatedAccuracy/numberOfItems, accumulatedLoss/numberOfItems))

In [132]:
accuracy(selfLearningModel(xValidationNormalized), yValidation)

tensor(0.0893)

In [133]:
trainModelWithValidation(selfLearningModel, imageDataBunch, 9)

Epoch 0, Accuracy 0.4959999918937683, Loss 1.2575938701629639
Epoch 1, Accuracy 0.5776000022888184, Loss 1.8194020986557007
Epoch 2, Accuracy 0.6467000246047974, Loss 1.2795933485031128
Epoch 3, Accuracy 0.8259999752044678, Loss 0.512549102306366
Epoch 4, Accuracy 0.8981999754905701, Loss 0.34089431166648865
Epoch 5, Accuracy 0.9060999751091003, Loss 0.2967222332954407
Epoch 6, Accuracy 0.91839998960495, Loss 0.2649284899234772
Epoch 7, Accuracy 0.9243999719619751, Loss 0.25057950615882874
Epoch 8, Accuracy 0.9247000217437744, Loss 0.2457549124956131


In [134]:
accuracy(selfLearningModel(xValidationNormalized), yValidation)

tensor(0.9247)

## Other Stuff

Things that facilitate the trainging of models

In [152]:
#export
class CNNModel(torch.nn.Module): 
    def __init__(self, inputSize, numberHiddenLayers, classes):
        super().__init__()
        self.layers = torch.nn.Sequential(
            torch.nn.Linear(inputSize, numberHiddenLayers),
            torch.nn.ReLU(),
            torch.nn.Linear(numberHiddenLayers, classes)
        )
        
    def __call__(self, inputMatrix): return self.layers(inputMatrix)


In [153]:
CNNModel(7,2,2)

CNNModel(
  (layers): Sequential(
    (0): Linear(in_features=7, out_features=2, bias=True)
    (1): ReLU()
    (2): Linear(in_features=2, out_features=2, bias=True)
  )
)

In [221]:
class Teacher():
    
    def __init__(self, 
                 lossFunction=Functional.cross_entropy, 
                 accuracyFunction=accuracy
                ):
        self.lossFunction = lossFunction
        self.accuracyFunction = accuracyFunction
        
    def teachModel(self, cnnModel, dataBunch, learningRate, numberOfEpochs):
        self.optimizer = optim.SGD(cnnModel.parameters(), learningRate)
        for epoch in range(numberOfEpochs):
            trainingLoss, trainingAccuracy = self._trainModel(cnnModel, dataBunch.trainingDataLoader)
            print("Epoch #{} Training: Loss {} Accuracy {}".format(epoch, trainingLoss, trainingAccuracy))
            
            validationLoss, validationAccuracy = self._validateModel(cnnModel, dataBunch.validationDataLoader)
            print("Epoch #{} Validation: Loss {} Accuracy {}".format(epoch, validationLoss, validationAccuracy))
            print("")
            
    def _trainModel(self, cnnModel, trainingDataSet):
        def _teachModel(loss):
            loss.backward()
            self.optimizer.step()
            self.optimizer.zero_grad()
            
        returnItems = self._proccessDataSet(cnnModel, trainingDataSet, _teachModel)
        return returnItems
        
    def _validateModel(self, cnnModel, validationDataSet):
        with torch.no_grad():
            returnItems = self._proccessDataSet(cnnModel, validationDataSet)
        return returnItems
    
    def _proccessDataSet(self, 
                         cnnModel, 
                         dataLoader, 
                         postEvaluation=lambda loss: None):
        accumulatedLoss, accumulatedAccuracy = 0.,0.
        for _xDataSet, _yDataSet in dataLoader:
            _predictions = cnnModel(_xDataSet)
            loss = self.lossFunction(_predictions, _yDataSet)
            postEvaluation(loss)
            accumulatedLoss+= loss
            accumulatedAccuracy += self.accuracyFunction(_predictions, _yDataSet)
        numberOfBatches = len(dataLoader)
        return accumulatedLoss/numberOfBatches, accumulatedAccuracy/numberOfBatches


In [222]:
teacher = Teacher()

In [223]:
cnnModel = CNNModel(trainingColumns, hiddenLayerOutput,numberOfClasses)
selfLearndingModel = SelfLearningLibraryModel(trainingColumns, hiddenLayerOutput,numberOfClasses,0.5)

In [224]:
teacher.teachModel(cnnModel, imageDataBunch, 0.5, 10)

Epoch #0 Training: Loss 1.974439263343811 Accuracy 0.3993000090122223
Epoch #0 Validation: Loss 1.2277991771697998 Accuracy 0.5770000219345093

Epoch #1 Training: Loss 1.1788398027420044 Accuracy 0.6037400364875793
Epoch #1 Validation: Loss 1.004588007926941 Accuracy 0.6527000069618225

Epoch #2 Training: Loss 0.6883408427238464 Accuracy 0.771340012550354
Epoch #2 Validation: Loss 0.5314133763313293 Accuracy 0.8228999972343445

Epoch #3 Training: Loss 0.477916955947876 Accuracy 0.83760005235672
Epoch #3 Validation: Loss 0.32999297976493835 Accuracy 0.8967999815940857

Epoch #4 Training: Loss 0.3720850944519043 Accuracy 0.8812600374221802
Epoch #4 Validation: Loss 0.2776019275188446 Accuracy 0.9178000092506409

Epoch #5 Training: Loss 0.29000285267829895 Accuracy 0.9121400713920593
Epoch #5 Validation: Loss 0.2502813935279846 Accuracy 0.9240999817848206

Epoch #6 Training: Loss 0.27402135729789734 Accuracy 0.9174400568008423
Epoch #6 Validation: Loss 0.2853277325630188 Accuracy 0.911599

In [225]:
trainModelWithValidation(selfLearndingModel, imageDataBunch, 10)

Epoch 0, Accuracy 0.4961000084877014, Loss 1.356858491897583
Epoch 1, Accuracy 0.728600025177002, Loss 0.7562280893325806
Epoch 2, Accuracy 0.8295999765396118, Loss 0.48906758427619934
Epoch 3, Accuracy 0.82669997215271, Loss 0.5604792237281799
Epoch 4, Accuracy 0.8892999887466431, Loss 0.3604641556739807
Epoch 5, Accuracy 0.9243000149726868, Loss 0.25576335191726685
Epoch 6, Accuracy 0.9254999756813049, Loss 0.25008153915405273
Epoch 7, Accuracy 0.9253000020980835, Loss 0.24883303046226501
Epoch 8, Accuracy 0.9329000115394592, Loss 0.22752465307712555
Epoch 9, Accuracy 0.9375, Loss 0.2182329297065735


In [226]:
accuracy(cnnModel(xValidationNormalized), yValidation), accuracy(selfLearndingModel(xValidationNormalized), yValidation)

(tensor(0.9428), tensor(0.9375))

In [227]:
!python notebook2script.py 04_HOLLA_BACK_GURL.ipynb

Converted 04_HOLLA_BACK_GURL.ipynb to exp/nb_04.py
